In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth
!pip install --upgrade transformers

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None          # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True   # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "thangvip/vilaw-sailor-instruct-v3",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.34G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

thangvip/vilaw-sailor-instruct-v3 does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,    # Supports any, but = 0 is optimized
    bias = "none",       # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [4]:
from transformers import AutoTokenizer
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_sharegpt

# Then apply the chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen-2.5",
)

def formatting_prompts_func(example):
    instructions = example["jp"]
    responses = example["vn"]
    texts = []
    for instruction, response in zip(instructions, responses): # Iterate over batches
        instruction = str(instruction) # Ensure string type
        response = str(response)       # Ensure string type
        messages = [
            {"role": "user", "content": f"Please translate this Japanese sentence into Vietnamese : {instruction}"},
            {"role": "assistant", "content": response},
        ]
        text = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=False)
        texts.append(text)
    example["text"] = texts # Assign list of texts to "text" key
    return example

from datasets import load_dataset
csv_path = "/kaggle/input/dataset-depvcl/batch2_merged.csv"
dataset = load_dataset("csv", data_files=csv_path)
dataset = dataset.map(formatting_prompts_func, batched = True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/501168 [00:00<?, ? examples/s]

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4, # Fixed major bug in latest Unsloth
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit", # Save more memory
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/501168 [00:00<?, ? examples/s]

In [6]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla P100-PCIE-16GB. Max memory = 15.888 GB.
2.422 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 501,168 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 14,991,360/1,243,508,736 (1.21% trained)


Step,Training Loss
1,3.019900
2,3.370200
3,3.561500
4,3.441500
5,3.057000
6,2.700200
7,2.638300
8,2.407600
9,2.390100
10,2.438600


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [8]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)

print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

60.483 seconds used for training.
1.01 minutes used for training.
Peak reserved memory = 2.422 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 15.244 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [9]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "translate this sequence from japanese into vietnamese: それは ローマ教皇庁の"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Đó là giáo hoàng của Lãnh thổ.<|im_end|>


In [10]:
from transformers import TextStreamer

# Enable faster inference
FastLanguageModel.for_inference(model)

# List of all 10 test cases with Japanese inputs and Vietnamese references
test_cases = [
    {
        "japanese": "経済成長を持続可能なものにするためには、環境保護と開発のバランスを取ることが不可欠である。特に気候変動の影響が顕著になる中で、各国は再生可能エネルギーへの移行を加速させている。しかし、この転換には多大なコストが伴い、発展途上国にとっては特に困難な課題となっている。",
        "vietnamese": "Để làm cho tăng trưởng kinh tế trở nên bền vững, việc cân bằng giữa bảo vệ môi trường và phát triển là điều cần thiết. Đặc biệt khi tác động của biến đổi khí hậu ngày càng rõ rệt, các quốc gia đang đẩy nhanh quá trình chuyển đổi sang năng lượng tái tạo. Tuy nhiên, quá trình chuyển đổi này đi kèm với chi phí lớn và trở thành thách thức đặc biệt khó khăn đối với các nước đang phát triển."
    },
    {
        "japanese": "人工知能の進化は私たちの生活を大きく変えつつあるが、同時に倫理的な問題も提起している。例えば、AIが人間の仕事を代替することで失業問題が深刻化する可能性や、アルゴリズムの偏りが社会的不平等を助長する危険性がある。",
        "vietnamese": "Sự tiến hóa của trí tuệ nhân tạo đang thay đổi cuộc sống của chúng ta một cách đáng kể, nhưng đồng thời cũng đặt ra những vấn đề đạo đức. Chẳng hạn, việc AI thay thế công việc của con người có thể làm trầm trọng thêm vấn đề thất nghiệp, hoặc sự thiên vị trong thuật toán có nguy cơ làm trầm trọng thêm bất bình đẳng xã hội."
    },
    {
        "japanese": "異文化間コミュニケーションにおいては、言語能力だけでなく、相手の文化的背景を理解する姿勢が重要である。些細なジェスチャーや表現が、異なる文化圏では全く別の意味に受け取られることがあるため、常に敏感である必要がある。",
        "vietnamese": "Trong giao tiếp liên văn hóa, không chỉ năng lực ngôn ngữ mà thái độ hiểu biết về bối cảnh văn hóa của đối phương cũng rất quan trọng. Những cử chỉ hay cách diễn đạt nhỏ nhặt có thể được hiểu theo nghĩa hoàn toàn khác trong các nền văn hóa khác nhau, vì vậy luôn cần phải nhạy cảm."
    },
    {
        "japanese": "現代社会では情報が氾濫しているため、真偽を見極める能力がますます重要になっている。特にソーシャルメディア上では誤情報が拡散しやすく、個人レベルでの情報リテラシーの向上が求められている。",
        "vietnamese": "Trong xã hội hiện đại, thông tin tràn ngập khiến khả năng phân biệt thật giả ngày càng trở nên quan trọng. Đặc biệt trên mạng xã hội, thông tin sai lệch dễ lan truyền, đòi hỏi phải nâng cao kiến thức thông tin ở cấp độ cá nhân."
    },
    {
        "japanese": "少子高齢化が進む日本では、労働力不足が深刻な問題となっている。この解決策として、外国人労働者の受け入れ拡大や、ロボット技術の活用などが検討されているが、根本的な解決には至っていない。",
        "vietnamese": "Ở Nhật Bản, nơi tình trạng già hóa dân số và tỷ lệ sinh thấp đang gia tăng, thiếu hụt lực lượng lao động đã trở thành vấn đề nghiêm trọng. Để giải quyết vấn đề này, việc mở rộng tiếp nhận lao động nước ngoài và ứng dụng công nghệ robot đang được xem xét, nhưng vẫn chưa có giải pháp triệt để."
    },
    {
        "japanese": "伝統的な価値観と現代的な考え方の間で葛藤を感じる若者が増えている。特にキャリア選択や結婚観において、個人の自由と家族の期待の板挟みになるケースが少なくない。",
        "vietnamese": "Ngày càng nhiều thanh niên cảm thấy xung đột giữa giá trị truyền thống và cách nghĩ hiện đại. Đặc biệt trong việc lựa chọn nghề nghiệp và quan niệm hôn nhân, không ít trường hợp bị kẹt giữa tự do cá nhân và kỳ vọng của gia đình."
    },
    {
        "japanese": "グローバル化が進む中で、多言語能力は単なるスキルではなく、異文化を理解するための重要なツールとなっている。言語学習を通じて得られるのはコミュニケーション手段だけでなく、その背後にある文化や歴史への洞察である。",
        "vietnamese": "Trong bối cảnh toàn cầu hóa phát triển, năng lực đa ngôn ngữ không chỉ là kỹ năng mà đã trở thành công cụ quan trọng để hiểu các nền văn hóa khác. Những gì đạt được thông qua việc học ngôn ngữ không chỉ là phương tiện giao tiếp mà còn là cái nhìn sâu sắc về văn hóa và lịch sử đằng sau nó."
    },
    {
        "japanese": "自然災害への備えは個人レベルから国家レベルまであらゆる段階で必要である。日本は地震や台風が多いため、防災訓練やインフラ整備が進んでいるが、気候変動の影響で想定外の災害が増える可能性がある。",
        "vietnamese": "Chuẩn bị ứng phó với thiên tai là cần thiết ở mọi cấp độ, từ cá nhân đến quốc gia. Do Nhật Bản thường xuyên xảy ra động đất và bão, các bài tập phòng chống thiên tai và xây dựng cơ sở hạ tầng đã được tiến hành, nhưng tác động của biến đổi khí hậu có thể làm tăng các thảm họa ngoài dự kiến."
    },
    {
        "japanese": "働き方改革が叫ばれるようになって久しいが、実際には長時間労働が根強く残っている企業も多い。真の改革のためには、労働者の意識改革だけでなく、経営陣の考え方を変える必要があるだろう。",
        "vietnamese": "Đã lâu kể từ khi cải cách cách làm việc được kêu gọi, nhưng trên thực tế vẫn còn nhiều công ty duy trì tình trạng làm việc quá giờ. Để có cải cách thực sự, không chỉ cần thay đổi nhận thức của người lao động mà còn phải thay đổi cách nghĩ của ban lãnh đạo."
    },
    {
        "japanese": "教育のデジタル化はコロナ禍で加速したが、対面授業とのバランスが課題となっている。特に発達段階にある子供たちにとって、直接的な人間関係を通じた学びはデジタルでは代替できない要素が多い。",
        "vietnamese": "Số hóa giáo dục đã tăng tốc trong đại dịch COVID-19, nhưng việc cân bằng với các lớp học trực tiếp đang trở thành vấn đề. Đặc biệt đối với trẻ em đang trong giai đoạn phát triển, có nhiều yếu tố học tập thông qua mối quan hệ trực tiếp giữa người với người mà phương pháp kỹ thuật số không thể thay thế."
    }
]

# Function to run translation tests
def run_translation_tests(model, tokenizer, test_cases):
    text_streamer = TextStreamer(tokenizer, skip_prompt=True)
    
    for i, test in enumerate(test_cases, 1):
        print(f"\n=== Test {i} ===")
        print("Japanese input:")
        print(test["japanese"])
        
        messages = [
            {"role": "user", "content": f"translate this sequence from japanese into vietnamese: {test['japanese']}"}, 
        ]
        
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")
        
        print("\nModel translation:")
        _ = model.generate(
            inputs=inputs, 
            streamer=text_streamer, 
            max_new_tokens=512,
            use_cache=True, 
            temperature=0.3, 
            min_p=0.1
        )
        
        print("\nReference translation:")
        print(test["vietnamese"])
        print("="*80)

# Run all tests
run_translation_tests(model, tokenizer, test_cases)


=== Test 1 ===
Japanese input:
経済成長を持続可能なものにするためには、環境保護と開発のバランスを取ることが不可欠である。特に気候変動の影響が顕著になる中で、各国は再生可能エネルギーへの移行を加速させている。しかし、この転換には多大なコストが伴い、発展途上国にとっては特に困難な課題となっている。

Model translation:
Có thể duy trì sự tăng trưởng bằng cách đảm bảo rằng môi trường và phát triển được cân bằng. Đặc biệt là tác động của biến đổi khí hậu ngày càng trở nên rõ rệt, các quốc gia đang nỗ lực chuyển sang các nguồn năng lượng tái tạo. Tuy nhiên, chuyển đổi này có thể gặp khó khăn về mặt chi phí, đặc biệt là đối với các quốc gia đang phát triển, nơi đây là một thách thức lớn.<|im_end|>

Reference translation:
Để làm cho tăng trưởng kinh tế trở nên bền vững, việc cân bằng giữa bảo vệ môi trường và phát triển là điều cần thiết. Đặc biệt khi tác động của biến đổi khí hậu ngày càng rõ rệt, các quốc gia đang đẩy nhanh quá trình chuyển đổi sang năng lượng tái tạo. Tuy nhiên, quá trình chuyển đổi này đi kèm với chi phí lớn và trở thành thách thức đặc biệt khó khăn đối với các nước đang phát triển.

=== Test 2 ===
Japan

In [11]:
# model.save_pretrained("Qwen2.5-3B-Instruct-KAI-jp-to-vi") # Local saving
# tokenizer.save_pretrained("Qwen2.5-3B-Instruct-KAI-jp-to-vi")

# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [12]:
# !zip -r file.zip /kaggle/working/Qwen2.5-3B-Instruct-KAI-jp-to-vi

In [13]:
# %% [code]
# Install required packages (only if not already installed)
try:
    from sacrebleu.metrics import BLEU
except ImportError:
    !pip install sacrebleu sacremoses
    from sacrebleu.metrics import BLEU

import pandas as pd
from tqdm import tqdm
import os

# %% [code]
def load_test_data(file_path="/kaggle/input/bleu-calculate/test_output.csv"):
    """Load and validate test data"""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Test file not found at {file_path}")
    
    test_df = pd.read_csv(file_path)
    
    # Verify required columns
    if not all(col in test_df.columns for col in ['jp_test', 'vn_test']):
        raise ValueError("CSV must contain 'jp_test' and 'vn_test' columns")
    
    return test_df

# %% [code]
def generate_translation(model, tokenizer, japanese_text):
    """Generate clean translation output"""
    messages = [
        {"role": "user", "content": f"translate this sequence from japanese into vietnamese: {japanese_text}"},
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)
    
    outputs = model.generate(
        inputs,
        max_new_tokens=512,
        use_cache=True,
        temperature=0.3,
        min_p=0.1,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Extract just the translation part
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_output.split("assistant\n")[-1].strip()

# %% [code]
def calculate_bleu(model, tokenizer, test_df, verbose=True):
    """Main evaluation function"""
    references = []
    hypotheses = []
    details = []
    
    for idx, (jp, vn) in tqdm(enumerate(zip(test_df["jp_test"], test_df["vn_test"])), 
                            total=len(test_df),
                            desc="Evaluating"):
        try:
            translation = generate_translation(model, tokenizer, jp)
            hypotheses.append(translation)
            references.append([vn])
            
            details.append({
                "id": idx,
                "japanese": jp,
                "reference": vn,
                "translation": translation,
                "reference_length": len(vn.split()),
                "translation_length": len(translation.split())
            })
            
            if verbose and idx < 5:  # Print first 5 examples
                print(f"\nExample {idx + 1}:")
                print(f"Japanese: {jp}")
                print(f"Reference: {vn}")
                print(f"Translation: {translation}")
                
        except Exception as e:
            print(f"Error processing sample {idx}: {str(e)}")
            continue
    
    # Calculate BLEU score
    bleu_score = BLEU(effective_order=True).corpus_score(hypotheses, references)
    
    # Calculate additional statistics
    details_df = pd.DataFrame(details)
    avg_ref_len = details_df["reference_length"].mean()
    avg_trans_len = details_df["translation_length"].mean()
    
    return {
        "bleu_score": bleu_score,
        "details": details_df,
        "stats": {
            "avg_reference_length": avg_ref_len,
            "avg_translation_length": avg_trans_len,
            "length_ratio": avg_trans_len / avg_ref_len
        }
    }

# %% [code]
def run_evaluation(model, tokenizer, test_file=None):
    """Complete evaluation workflow"""
    # Load data
    test_df = load_test_data(test_file if test_file else "/kaggle/input/bleu-calculate/test_output.csv")
    
    # Run evaluation
    results = calculate_bleu(model, tokenizer, test_df)
    
    # Print summary
    print("\n" + "="*60)
    print(f"BLEU Score: {results['bleu_score'].score:.2f}")
    print(f"Details: {results['bleu_score']}")
    print("\nStatistics:")
    print(f"Average reference length: {results['stats']['avg_reference_length']:.1f} words")
    print(f"Average translation length: {results['stats']['avg_translation_length']:.1f} words")
    print(f"Length ratio (translation/reference): {results['stats']['length_ratio']:.2f}")
    print("="*60 + "\n")
    
    # Save results
    results['details'].to_csv("translation_evaluation_details.csv", index=False)
    print("Saved detailed results to 'translation_evaluation_details.csv'")
    
    return results

# %% [code]
# Usage example (assuming model and tokenizer are already loaded):


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 28.4 MB/s eta 0:00:00


In [14]:
results = run_evaluation(model, tokenizer)

Evaluating:   0%|          | 1/1098 [00:00<16:53,  1.08it/s]


Example 1:
Japanese: ラジオの音を少し大きくしてよ。
Reference: Bật radio to lên một chút đi.
Translation: Khi tôi nghe radio, tôi có thể nghe được âm thanh.


Evaluating:   0%|          | 2/1098 [00:01<12:10,  1.50it/s]


Example 2:
Japanese: トムがどこに住んでいるか分かった。
Reference: Tôi biết Tom sống ở đâu.
Translation: Tomo đang ở đâu không biết.


Evaluating:   0%|          | 3/1098 [00:02<13:50,  1.32it/s]

...


Example 4:
Japanese: 私の物理の先生は授業をサボっても気にしない。
Reference: Thầy vật lý không để ý đến việc lên lớp của tôi.
Translation: Tôi không quan tâm đến việc giáo viên của mình đang làm gì.


Evaluating:   0%|          | 5/1098 [00:03<11:43,  1.55it/s]


Example 5:
Japanese: 気づいてた？
Reference: Bạn có nhìn thấy không?
Translation: 「気づいてた？


Evaluating: 100%|██████████| 1098/1098 [13:39<00:00,  1.34it/s]


BLEU Score: 28.30
Details: BLEU = 28.30 84.6/41.7/18.2/10.0 (BP = 1.000 ratio = 1.000 hyp_len = 13 ref_len = 13)

Statistics:
Average reference length: 8.6 words
Average translation length: 9.9 words
Length ratio (translation/reference): 1.15

Saved detailed results to 'translation_evaluation_details.csv'
